In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

keywords = {
    # MBTI 카테고리 별 키워드 선정
    'E': ["외향", "사교", "활동적", "야외", "모험", "경험", "스릴", "팀워크", "도전", "자극", "열정", "인간관계", "소통", "친화력", "활발"],
    'I': ["내향", "차분함", "집중", "실내", "조용", "몰입", "혼자", "상상력", "독서", "사색", "사색", "개인적", "자기 성찰", "자아 발견"],
    'N': ["생각", "직관", "창의성", "아이디어", "미래지향", "영감", "호기심", "상상", "추상", "비전", "독창성", "미래", "첨단 기술", "혁신"],
    'S': ["기분", "느낌", "감각", "구체적", "실용적", "세부 사항", "경험", "관찰", "현실적", "실천", "관행", "실제", "실증", "실질적", "현실", "사실"],
    'F': ["감정", "공감", "협력", "인간 관계", "감수성", "감성", "관계 지향", "배려", "자연", "따뜻함", "인화", "친밀함", "애정", "사랑", "정성"],
    'T': ["사고", "논리적", "분석적", "객관적", "효율성", "문제 해결", "비판적 사고", "합리성", "이성", "관념적", "논리", "구조", "체계", "분석", "이해"],
    'J': ["오래", "판단", "계획", "조직적", "결정적", "구조화", "목표 지향", "책임감", "규칙", "목표", "결심", "지도력", "리더십", "체계", "정리", "결정"],
    'P': ["인식", "유연성", "즉흥", "자유로움", "개방성", "적응력", "자율성", "탐구", "탐험", "융통성", "유연", "변화", "자유", "탐구", "독립"],
}

data_crawling = {
    # 각 취미 별 크롤링한 문장
    '자전거': ["레크리에이션", "스포츠", "건강", "자유", "야외", "여유", "모험", "활동적", "기분", "감성", "느낌", "즉흥"],
    '캠핑': ["자연", "야외", "여가", "경험", "체험", "즐거운", "평온함", "안정", "여유", "계획", "감성", "감정", "기분"],
    '골프': ["건강", "집중력", "정밀함", "경치", "소통", "야외", "기분", "생각", "이성", "계획"],
    '등산': ["자연", "운동", "모험", "성취감", "경치", "힐링", "야외", "기분", "느낌", "감성", "계획"],
    '크로스핏': ["운동", "근력", "체력", "도전", "지구력", "집중", "열정", "실내", "계획", "생각", "이성"],
    '서핑': ["바다", "모험", "스릴", "스포츠", "자유", "여름", "어드벤처", "에너지", "야외", "즉흥", "기분", "감정", "감각"],
    '테니스': ["운동", "경쟁", "민첩성", "전략", "소통", "야외", "집중", "끈기", "야외", "기분", "즉흥", "이성"],
    '스케이트보드': ["스포츠", "도전", "균형", "스타일", "기술", "액션", "창의성", "모험심", "야외", "감성", "기분", "즉흥"],
    '스키': ["겨울", "경치", "스릴", "기술", "스포츠", "어드벤처", "에너지", "야외", "계획", "기분", "감성"],
    '볼링': ["실내", "정확성", "사교", "즐거움", "경쟁", "오락", "집중", "전략적", "즉흥", "생각", "이성"],
    '산책': ["여유", "자연", "건강", "마음의 평화", "휴식", "운동", "힐링", "평온함", "야외", "감성", "즉흥", "기분"],
    '클라이밍': ["도전", "모험", "높이", "성취감", "집중력", "체력", "끈기", "모험심", "야외", "계획", "생각", "상상", "사고", "분석적"],
    '풋살': ["팀워크", "경쟁", "축구", "운동", "스포츠", "사교", "에너지", "전략적", "야외", "즉흥", "생각", "이성"],
    '필라테스': ["유연성", "건강", "평화", "집중", "힐링", "차분함", "실내", "기분", "느낌", "계획", "감수성"],
    '배드민턴': ["운동", "경쟁", "민첩성", "스피드", "야외", "사교", "에너지", "전략적", "즉흥", "기분", "협력", "인간 관계"],
    '러닝': ["운동", "지구력", "건강", "스트레스 해소", "야외", "목표", "에너지", "끈기", "계획", "생각", "이성"],
    '사진': ["창의성", "기록", "예술", "기술", "자연", "경험", "관찰력", "표현력", "야외", "실내", "감성", "감정", "계획"],
    '미술': ["창의성", "표현", "예술", "기술", "평온함", "몰입", "집중", "감성적", "실내", "상상", "계획", "상상", "감정", "공감", "감수성"],
    '캘리그라피': ["감정", "따뜻함", "예술", "글씨", "창의성", "집중", "평온함", "표현", "섬세함", "차분함", "실내", "느낌", "즉흥"],
    '전시회': ["예술", "문화", "영감", "경험", "창의성", "감상", "호기심", "지적", "실내", "계획", "감성", "생각", "상상"],
    '연극': ["문화", "표현", "연기", "창의성", "팀워크", "즐거움", "사교", "감성적", "실내", "기분", "느낌", "김성", "공감", "감정", "즉흥"],
    '가죽 공예': ["창의성", "손작업", "예술", "기술", "집중", "실용성", "섬세함", "차분함", "실내", "생각", "계획", "이성"],
    '레고 조립': ["창의성", "조립", "취미", "상상", "집중", "완성", "논리적", "놀이", "실내", "계획", "생각"],
    '인테리어': ["감정", "감성", "창의성", "디자인", "공간", "꾸미기", "취향", "실용성", "미적 감각", "개성", "실내", "계획", "생각"],
    '뜨개질': ["감정", "따뜻함", "정교함", "집중", "창의성", "휴식", "만들기", "완성", "섬세함", "차분함", "실내", "생각", "상상", "계획"],
    '반려식물': ["자연", "힐링", "책임감", "인테리어", "생명", "여유", "보살핌", "정서적 안정", "실내", "오래", "감성", "감정", "기분"],
    '홈브루': ["계획", "요리","창의성", "실험", "완성", "실용적", "정밀함", "실내", "맛", "기분", "느낌", "이성"],
    '베이킹': ["감성", "정성","요리","창의성", "맛", "과정", "즐거움", "완성", "섬세함", "실내", "기분", "느낌", "상상", "계획"],
    '홈카페': ["요리","창의성", "맛", "여유", "휴식", "실내", "즉흥", "기분", "느낌", "감성"],
    '바베큐': ["자연", "감성", "요리","야외", "친구", "즐거움", "맛", "사교", "실외", "기분", "느낌", "계획"],
    '코딩': ["기술", "창의성", "문제 해결", "논리", "학습", "실용성", "실내", "탐구", "생각", "상상", "계획"],
    '천체관측': ["과학", "자연", "관찰", "평온함", "지식", "야외", "호기심", "탐구", "계획", "생각", "계획"],
    '보드 게임': ["전략", "사교", "즐거움", "경쟁", "창의성", "집중", "실내", "즉흥", "생각", "상상","감정"],
    '비디오 게임': ["엔터테인먼트", "전략", "몰입", "기술", "경쟁", "창의성", "실내", "즉흥", "기분", "느낌", "감정"],
    '영화': ["엔터테인먼트", "예술", "스토리", "감정", "감상", "문화", "실내", "감성", "공감", "상상", "기분", "개방성"],
    '마술': ["논리적", "사고", "엔터테인먼트", "창의성", "기술", "몰입", "놀라움", "퍼포먼스", "실내", "야외", "계획", "생각", "상상"],
    '낚시': ["자연", "휴식", "스릴", "기술", "인내", "야외", "평온", "기분", "느낌", "즉흥"],
    '여행': ["모험", "경험", "자유", "문화", "힐링", "탐험", "자연", "야외", "계획", "즉흥", "기분", "느낌", "감성"],
    '음악 감상': ["예술", "휴식", "감성", "문화", "즐거움", "몰입", "실내", "야외", "기분", "느낌", "감정", "즉흥"],
    '타로': ["신비", "예언", "심리", "상담", "몰입", "자기 성찰", "실내", "즉흥", "기분", "감정", "감성"],
    '나무 심기': ["자연", "환경", "사회 기여", "야외", "책임감", "지속 가능성", "야외", "계획", "기분", "느낌", "감정"],
    '낭독 봉사': ["사회 기여", "책임감", "소통", "문화", "도움", "감동", "실내", "계획", "기분", "느낌", "계획", "감성", "공감","김장"],
    '유기견 봉사': ["동물 사랑", "책임감", "사회 기여", "사랑", "헌신", "돌봄", "야외", "계획", "감정", "공감", "감성", "생각"],
    '베이비박스 봉사': ["사회 기여", "책임감", "사랑", "돌봄", "도움", "헌신", "활동", "실내", "계획", "생각", "감성", "공감", "감정"],
    '플로깅': ["환경", "운동", "자연", "책임감", "건강", "기여", "야외", "활동", "계획", "기분", "느낌", "즉흥"],
}


# 유사도 판단 함수
def get_score(hobby, category):
    hobby_sentences = data_crawling[hobby]
    category_keywords = keywords[category]
    s_len = len(hobby_sentences)
    # 취미와 카테고리의 키워드를 각각 하나의 문자열로 결합
    hobby_text = " ".join(hobby_sentences)
    category_text = " ".join(category_keywords)

    # TF-IDF 행렬 생성을 위해 리스트로 문장을 묶음
    sentences = [category_text, hobby_text]
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
    val = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).tolist()[0]
    val.sort(reverse=True)
    total = s_len - val.count(0.0)
    try:
        return sum(val)/total
    except:
        return 0

hobby_list = list(data_crawling.keys())
all_hobby_similarity = {}

for hobby in hobby_list:
    hobby_similarity = {}
    for category in ['E', 'I', 'N', 'S', 'F', 'T', 'J', 'P']:
        score = get_score(hobby, category)
        if score > 0:  # 유사도가 0보다 큰 경우에만 딕셔너리에 추가
            hobby_similarity[category] = score

    # E와 I 비교하여 더 높은 값으로 대체
    if 'E' in hobby_similarity and 'I' in hobby_similarity:
        if hobby_similarity['E'] > hobby_similarity['I']:
            hobby_similarity['E'] = hobby_similarity['E']
            del hobby_similarity['I']
        else:
            hobby_similarity['I'] = hobby_similarity['I']
            del hobby_similarity['E']

    # N와 S 비교하여 더 높은 값으로 대체
    if 'N' in hobby_similarity and 'S' in hobby_similarity:
        if hobby_similarity['N'] > hobby_similarity['S']:
            hobby_similarity['N'] = hobby_similarity['N']
            del hobby_similarity['S']
        else:
            hobby_similarity['S'] = hobby_similarity['S']
            del hobby_similarity['N']

    # F와 T 비교하여 더 높은 값으로 대체
    if 'F' in hobby_similarity and 'T' in hobby_similarity:
        if hobby_similarity['F'] > hobby_similarity['T']:
            hobby_similarity['F'] = hobby_similarity['F']
            del hobby_similarity['T']
        else:
            hobby_similarity['T'] = hobby_similarity['T']
            del hobby_similarity['F']

    # J와 P 비교하여 더 높은 값으로 대체
    if 'J' in hobby_similarity and 'P' in hobby_similarity:
        if hobby_similarity['J'] > hobby_similarity['P']:
            hobby_similarity['J'] = hobby_similarity['J']
            del hobby_similarity['P']
        else:
            hobby_similarity['P'] = hobby_similarity['P']
            del hobby_similarity['J']

    all_hobby_similarity[hobby] = hobby_similarity

# 결과 출력
for hobby, similarity in all_hobby_similarity.items():
    print(f"{hobby} = {similarity}")

자전거 = {'E': 0.010613294149566127, 'S': 0.006353843040259484, 'F': 0.002891030587123223, 'P': 0.006353843040259482}
캠핑 = {'E': 0.006003056051186024, 'S': 0.005615657555394777, 'F': 0.008224018851202528, 'J': 0.0025597286014074974}
골프 = {'E': 0.009009690756912405, 'N': 0.0043109987479869535, 'T': 0.003816772230154735, 'J': 0.003816772230154735}
등산 = {'E': 0.007770850340357813, 'S': 0.007269369809907203, 'F': 0.006853924550400398, 'J': 0.0033005346448451464}
크로스핏 = {'E': 0.007770850340357813, 'N': 0.0037279145475857104, 'T': 0.0033005346448451464, 'J': 0.0033005346448451464}
서핑 = {'E': 0.009361974473807804, 'S': 0.005615657555394777, 'F': 0.0025597286014074974, 'P': 0.005615657555394777}
테니스 = {'E': 0.00995645235673873, 'S': 0.002825447175548655, 'T': 0.0026684076711426543, 'P': 0.002825447175548654}
스케이트보드 = {'E': 0.006792165571861426, 'N': 0.006792165571861426, 'F': 0.002891030587123223, 'P': 0.00306117175990362}
스키 = {'E': 0.007770850340357813, 'N': 0.0037279145475857104, 'F': 0.003300

In [ ]:
# 각 카테고리별 최대 유사도 값을 찾기
max_scores = {category: 0 for category in ['E', 'I', 'N', 'S', 'F', 'T', 'J', 'P']}

for hobby, similarity in all_hobby_similarity.items():
    for category, score in similarity.items():
        if score > max_scores[category]:
            max_scores[category] = score

# 0~1점으로 정규화
normalized_all_hobby_similarity = {}

for hobby, similarity in all_hobby_similarity.items():
    normalized_similarity = {}
    for category, score in similarity.items():
        if max_scores[category] > 0:  # 0으로 나누는 것을 방지
            normalized_score = (score / max_scores[category])
        else:
            normalized_score = 0
        normalized_similarity[category] = normalized_score
    normalized_all_hobby_similarity[hobby] = normalized_similarity

# 결과 출력
for hobby, similarity in normalized_all_hobby_similarity.items():
    print(f"{hobby} = {similarity}")

자전거 = {'E': 1.0, 'S': 0.6101748080408217, 'F': 0.2002904682160102, 'P': 0.7271586372091667}
캠핑 = {'E': 0.5656166659087113, 'S': 0.5392850829292853, 'F': 0.5697596537585305, 'J': 0.2501500891303888}
골프 = {'E': 0.8489061576872174, 'N': 0.25434745455900004, 'T': 0.37526739924190133, 'J': 0.3729949780764298}
등산 = {'E': 0.7321808131244046, 'S': 0.6980950426746343, 'F': 0.4748395826150392, 'J': 0.32254553671509056}
크로스핏 = {'E': 0.7321808131244046, 'N': 0.21994568577285079, 'T': 0.32451060152169925, 'J': 0.32254553671509056}
서핑 = {'E': 0.8820988414978136, 'S': 0.5392850829292853, 'F': 0.17733788164170972, 'P': 0.6426778044626459}
테니스 = {'E': 0.938111411634224, 'S': 0.27133447852679027, 'T': 0.2623594876727151, 'P': 0.3233552205586973}
스케이트보드 = {'E': 0.6399677118285739, 'N': 0.4007354502139272, 'F': 0.2002904682160102, 'P': 0.3503324635327784}
스키 = {'E': 0.7321808131244046, 'N': 0.21994568577285079, 'F': 0.228660890799154, 'J': 0.32254553671509056}
볼링 = {'I': 0.48557402479801975, 'N': 0.219945

In [ ]:
{
'자전거' : [3,1,2,1],
'캠핑' : [3,1,3,2],
'골프' : [3,2,0,2],
'등산' : [3,1,2,2],
'크로스핏' : [3,2,0,2],
'서핑' : [3,1,2,1],
'테니스' : [3,0,0,0],
'스케이트보드' : [3,2,2,0],
'스키' : [3,2,2,2],
'볼링' : [0,2,0,0],
'산책' : [2,0,2,0],
'클라이밍' : [2,2,1,2],
'풋살' : [3,2,0,0],
'필라테스' : [1,1,2,0],
'배드민턴' : [3,0,3,0],
'러닝' : [2,2,0,3],
'사진' : [3,2,3,2],
'미술' : [0,3,2,2],
'캘리그라피' : [1,2,2,0],
'전시회' : [2,3,2,2],
'연극' : [2,0,2,0],
'가죽공예' : [1,3,0,2],
'레고조립' : [0,3,0,2],
'인테리어' : [0,2,2,2],
'뜨개질' : [1,2,2,2],
'반려식물' : [0,0,3,2],
'홈브루' : [0,1,0,2],
'베이킹' : [0,0,2,2],
'홈카페' : [0,1,2,0],
'바베큐' : [3,1,2,2],
'코딩' : [0,3,1,1],
'천체관측' : [2,2,2,1],
'보드게임' : [0,3,2,0],
'비디오게임' : [0,2,2,0],
'영화' : [0,2,3,0],
'마술' : [0,3,1,2],
'낚시' : [3,1,2,0],
'여행' : [3,1,2,1],
'음악감상' : [0,1,2,0],
'타로' : [1,0,2,0],
'나무심기' : [3,1,2,3],
'낭독봉사' : [2,0,2,3],
'유기견봉사' : [2,2,3,3],
'베이비박스봉사' : [0,2,3,2],
'플로깅' : [2,1,2,3],
}

{'자전거': [3, 1, 2, 1],
 '캠핑': [3, 1, 3, 2],
 '골프': [3, 2, 0, 2],
 '등산': [3, 1, 2, 2],
 '크로스핏': [3, 2, 0, 2],
 '서핑': [3, 1, 2, 1],
 '테니스': [3, 0, 0, 0],
 '스케이트보드': [3, 2, 2, 0],
 '스키': [3, 2, 2, 2],
 '볼링': [0, 2, 0, 0],
 '산책': [2, 0, 2, 0],
 '클라이밍': [2, 2, 1, 2],
 '풋살': [3, 2, 0, 0],
 '필라테스': [1, 1, 2, 0],
 '배드민턴': [3, 0, 3, 0],
 '러닝': [2, 2, 0, 3],
 '사진': [3, 2, 3, 2],
 '미술': [0, 3, 2, 2],
 '캘리그라피': [1, 2, 2, 0],
 '전시회': [2, 3, 2, 2],
 '연극': [2, 0, 2, 0],
 '가죽공예': [1, 3, 0, 2],
 '레고조립': [0, 3, 0, 2],
 '인테리어': [0, 2, 2, 2],
 '뜨개질': [1, 2, 2, 2],
 '반려식물': [0, 0, 3, 2],
 '홈브루': [0, 1, 0, 2],
 '베이킹': [0, 0, 2, 2],
 '홈카페': [0, 1, 2, 0],
 '바베큐': [3, 1, 2, 2],
 '코딩': [0, 3, 1, 1],
 '천체관측': [2, 2, 2, 1],
 '보드게임': [0, 3, 2, 0],
 '비디오게임': [0, 2, 2, 0],
 '영화': [0, 2, 3, 0],
 '마술': [0, 3, 1, 2],
 '낚시': [3, 1, 2, 0],
 '여행': [3, 1, 2, 1],
 '음악감상': [0, 1, 2, 0],
 '타로': [1, 0, 2, 0],
 '나무심기': [3, 1, 2, 3],
 '낭독봉사': [2, 0, 2, 3],
 '유기견봉사': [2, 2, 3, 3],
 '베이비박스봉사': [0, 2, 3, 2],
 '플로깅': [2, 1, 2, 3]}

In [ ]:
{
  "hobby_enum": {
    "0": "자전거",
    "1": "캠핑",
    "2": "골프",
    "3": "등산",
    "4": "크로스핏",
    "5": "서핑",
    "6": "테니스",
    "7": "스케이트보드",
    "8": "스키",
    "9": "볼링",
    "10": "산책",
    "11": "클라이밍",
    "12": "풋살",
    "13": "필라테스",
    "14": "배드민턴",
    "15": "러닝",
    "16": "사진",
    "17": "미술",
    "18": "캘리그라피",
    "19": "전시회",
    "20": "연극",
    "21": "가죽공예",
    "22": "레고조립",
    "23": "인테리어",
    "24": "뜨개질",
    "25": "반려식물",
    "26": "홈브루",
    "27": "베이킹",
    "28": "홈카페",
    "29": "바베큐",
    "30": "코딩",
    "31": "천체관측",
    "32": "보드게임",
    "33": "비디오게임",
    "34": "영화",
    "35": "마술",
    "36": "낚시",
    "37": "여행",
    "38": "음악감상",
    "39": "타로",
    "40": "나무심기",
    "41": "낭독봉사",
    "42": "유기견봉사",
    "43": "베이비박스봉사",
    "44": "플로깅",
  },
  "list": [4, 4, 4, 4]
}

{'hobby_enum': {'0': '자전거',
  '1': '캠핑',
  '2': '골프',
  '3': '등산',
  '4': '크로스핏',
  '5': '서핑',
  '6': '테니스',
  '7': '스케이트보드',
  '8': '스키',
  '9': '볼링',
  '10': '산책',
  '11': '클라이밍',
  '12': '풋살',
  '13': '필라테스',
  '14': '배드민턴',
  '15': '러닝',
  '16': '사진',
  '17': '미술',
  '18': '캘리그라피',
  '19': '전시회',
  '20': '연극',
  '21': '가죽공예',
  '22': '레고조립',
  '23': '인테리어',
  '24': '뜨개질',
  '25': '반려식물',
  '26': '홈브루',
  '27': '베이킹',
  '28': '홈카페',
  '29': '바베큐',
  '30': '코딩',
  '31': '천체관측',
  '32': '보드게임',
  '33': '비디오게임',
  '34': '영화',
  '35': '마술',
  '36': '낚시',
  '37': '여행',
  '38': '음악감상',
  '39': '타로',
  '40': '나무심기',
  '41': '낭독봉사',
  '42': '유기견봉사',
  '43': '베이비박스봉사',
  '44': '플로깅'},
 'list': [4, 4, 4, 4]}